In [ ]:
%cd template

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Constants
data_file_path = "./data/home-data-for-ml-course/train.csv"
test_size = 0.2
val_size = 0.2
random_state = 0

# Load data
df = pd.read_csv(data_file_path)

# Target and features
y = df.SalePrice

# All numeric without missing values
features_no_missing = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
features = features_no_missing + ['LotFrontage','MasVnrArea','GarageYrBlt'] # numeric types with missing columns
X = df[features]

# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=random_state)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=val_size, random_state=random_state)

In [ ]:
# Number of missing values per column
missing_value_count_by_column = train_X.isnull().sum()
columns_with_missing_values = missing_value_count_by_column[missing_value_count_by_column > 0]
print(train_X.shape)
print(columns_with_missing_values)

In [ ]:
# Handle missing values
# Option 1: Drop columns with missing values. Not so great if the column to be dropped has a lot of data.
missing_handled_train_X = train_X.drop(columns_with_missing_values.index, axis=1)
missing_handled_valid_X = val_X.drop(columns_with_missing_values.index, axis=1)

# Note the difference in columns
print(train_X.columns)
print(missing_handled_train_X.columns)

In [ ]:
# Option 2: If not dropping the column, it's possible to replace missing values with the mean of that column.
# Other methods exist such as filling in with 0 or with the mode.
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
mean_imputed_train_X = pd.DataFrame(my_imputer.fit_transform(train_X), columns=train_X.columns)
mean_imputed_val_X = pd.DataFrame(my_imputer.transform(val_X), columns=val_X.columns)

# Note the difference in row counts of missing values
print("Number of rows in original data: " +
      str(train_X.shape[0]))
print("Number of rows with missing entries in original data: "
      + str(train_X[columns_with_missing_values.index].isnull().any(axis=1).sum()))
print("Number of rows in imputed data: " +
      str(mean_imputed_train_X.shape[0]))
print("Number of rows with missing entries in inputed data: "
      + str(mean_imputed_train_X[columns_with_missing_values.index].isnull().any(axis=1).sum()))

In [ ]:
# Option 3: Impute data while keeping track of entries that were imputed by creating a new column.
def track_missing_entries_and_impute(dfin):
    dfin = dfin.copy()
    for column in columns_with_missing_values.index:
        dfin[column + "_missing"] = dfin[column].isnull()
    return dfin

tracked_missing_train_X = track_missing_entries_and_impute(train_X)
tracked_missing_val_X = track_missing_entries_and_impute(val_X)
my_imputer = SimpleImputer()
imputed_tracked_missing_train_X = pd.DataFrame(my_imputer.fit_transform(tracked_missing_train_X), columns=tracked_missing_train_X.columns)
imputed_tracked_missing_val_X = pd.DataFrame(my_imputer.transform(tracked_missing_val_X), columns=tracked_missing_val_X.columns)

In [ ]:
# Compare different methods of handling missing values by computing MAE
def score(train_X, val_X, train_y, val_y):
    mdl = RandomForestRegressor(random_state=random_state)
    mdl.fit(train_X, train_y)
    mae = mean_absolute_error(val_y, mdl.predict(val_X))
    return (mae, mdl)

mae_regular, _ = score(train_X, val_X, train_y, val_y)
mae_imputed, _ = score(mean_imputed_train_X, mean_imputed_val_X, train_y, val_y)
mae_missing_tracked, _ = score(imputed_tracked_missing_train_X, imputed_tracked_missing_val_X, train_y, val_y)
print(f"mae_regular = {mae_regular}")
print(f"mae_imputed = {mae_imputed}")
print(f"mae_missing_tracked = {mae_missing_tracked}")

In [ ]:
# In addition to the above, other options include:
# 1. Impute using the mean/median/mode
# 2. Replace with a constant value
# 3. Replace with the mean/media/mode of a subgroup by using groupby
# 4. Use a machine learning model to predict the missing values
# 5. Use knn to impute the missing values